In [3]:
import os
import numpy as np
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D, AveragePooling2D, Flatten, Dense
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from tqdm import tqdm

# Define LeNet
def lenet_model(num_classes):
    model = Sequential()
    model.add(Conv2D(6, (5, 5), activation='relu', input_shape=(32, 32, 3)))
    model.add(AveragePooling2D((2, 2)))
    model.add(Conv2D(16, (5, 5), activation='relu'))
    model.add(AveragePooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(120, activation='relu'))
    model.add(Dense(84, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    return model

# Caminho base
root_directory = r'MED115\(sub)_MED_115_database_classes'
classes = os.listdir(root_directory)

# Modelo (não usado no pipeline atual, mas mantido se quiser treinar depois com Keras)
model = lenet_model(len(classes))

X = []
y = []

# Coleta todos os caminhos de imagem
all_image_paths = []
for class_name in classes:
    class_dir = os.path.join(root_directory, class_name)
    for filename in os.listdir(class_dir):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            all_image_paths.append((class_name, os.path.join(class_dir, filename)))

# Processa imagens com barra de progresso
for class_name, img_path in tqdm(all_image_paths, desc="Processando imagens"):
    try:
        img = image.load_img(img_path, target_size=(32, 32))
        img = image.img_to_array(img)
        img /= 255.0
        X.append(img)
        y.append(class_name)
    except Exception as e:
        print(f"Erro ao processar {img_path}: {e}")

X = np.array(X)
y = np.array(y)

# Treino/teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Flatten para Random Forest
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

# Random Forest
print("Treinando Random Forest...")
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_flat, y_train)

# Avaliação
print("Avaliando modelo...")
y_pred = clf.predict(X_test_flat)

accuracy = accuracy_score(y_test, y_pred) * 100
f1 = f1_score(y_test, y_pred, average='weighted') * 100
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0) * 100
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0) * 100

print(f"\nAcurácia: {accuracy:.2f}%")
print(f"F1 Score: {f1:.2f}%")
print(f"Precision: {precision:.2f}%")
print(f"Recall: {recall:.2f}%")


Processando imagens: 100%|█████████████████████████████████████████████████████████| 8809/8809 [01:31<00:00, 95.84it/s]


Treinando Random Forest...
Avaliando modelo...

Acurácia: 96.14%
F1 Score: 96.06%
Precisão: 96.25%
Recall: 96.14%
